In [1]:
import sys
sys.path.append('src')
import trie, utils, rlp

In [2]:
def convert_to_hex_escape(x):
    #used to force it to not print the char asscociated with that hex
    def converter(s):
        return ''.join('\\x{:02x}'.format(ord(c)) for c in s)
    converted_value = []
    if type(x) == list:
        for i in x:
            if type(i) == list:
                converted_value.append([converter(i[0]),i[1]])
            else:
                converted_value.append(converter(i))

        #check if it is a branch node
        if len(x) == 17:
            return converted_value[:-1] + [x[-1]]
        else:
            return converted_value
    else:
        return converter(x)


# Ex1
add one leaf node

In [3]:
#create empty trie
state = trie.Trie('trie_db', trie.BLANK_ROOT)

#add key-value pair (010101, 'hello')
state.update('\x01\x01\x01', 'hello')

k, v = state.root_node

#check the encoded key
print '\nhp encoded key, in hex:\n', k.encode('hex')

#check node at the root
print '\nroot node:\n', [k, v]
print 'key hex escape form:', convert_to_hex_escape(k)

#check root hash
print '\nroot hash:\n', state.root_hash.encode('hex')


hp encoded key, in hex:
20010101

root node:
[' \x01\x01\x01', 'hello']
key hex escape form: \x20\x01\x01\x01

root hash:
60ea522a5d21d6e5f16bb42de67409733c9dd5ed321cd34e82e26fe268ebcc62


In [4]:
print utils.sha3(rlp.encode(["\x20\x01\x01\x01", v])).encode('hex')
print utils.sha3(rlp.encode([" \x01\x01\x01", v])).encode('hex')

60ea522a5d21d6e5f16bb42de67409733c9dd5ed321cd34e82e26fe268ebcc62
60ea522a5d21d6e5f16bb42de67409733c9dd5ed321cd34e82e26fe268ebcc62


# Ex2
add another node that share 0101 path with the above example node but try to make it short so that it doesn't get stored with hash

In [5]:
#initialize trie like in Ex1
state = trie.Trie('trie_db', trie.BLANK_ROOT)

#add key-value pair (010101, 'hello')
state.update('\x01\x01\x01', 'hello')

#check that the root is matched with the Ex1
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node

root hash 60ea522a5d21d6e5f16bb42de67409733c9dd5ed321cd34e82e26fe268ebcc62


In [6]:
# add '\x01\x01\x82\x11', 'w' key-value pair to the trie
state.update('\x01\x01\x82\x11', 'w')

print '---------AFTER UPDATE----------'
k, v = state.root_node
#check the encoded key
print '\nhp encoded key, in hex:\n', k.encode('hex')

#check node at the root
# convert_to_hex_escape is a function that help prevent \xHH hexadecimal to be converted to its asscociate char
# for example "\x56" will be converted to "V" upon printed 
# convert_to_hex_escape is put here to force "\x56" to be printed as "\\x56" not "V"
print '\nroot node:\n', [k, convert_to_hex_escape(v)]
print 'key hex escape form:', convert_to_hex_escape(k)

#check root hash
print '\nroot hash:\n', state.root_hash.encode('hex')

---------AFTER UPDATE----------

hp encoded key, in hex:
000101

root node:
['\x00\x01\x01', [['\\x31', 'hello'], '', '', '', '', '', '', '', ['\\x32\\x11', 'w'], '', '', '', '', '', '', '', '']]
key hex escape form: \x00\x01\x01

root hash:
6f52a5679459075d2debfb2f5c0c4b7845d60c84b9154b2bcb9dd605734ea2c3


In [7]:
#print the branch node stored within the extension node
print '\nvalue stored within the extension node:\n', convert_to_hex_escape(v)

#print the lenght of the RLP encoded of the value with in the extension node
print '\nlenght of the RLP encoded value:\n', len(rlp.encode(v))



value stored within the extension node:
[['\\x31', 'hello'], '', '', '', '', '', '', '', ['\\x32\\x11', 'w'], '', '', '', '', '', '', '', '']

lenght of the RLP encoded value:
29


# Ex3
The same as the above example but make the added node long enough so that the branch node is stored using hash

In [8]:
#initialize trie like in Ex1
state = trie.Trie('trie_db', trie.BLANK_ROOT)

#add key-value pair (010101, 'hello')
state.update('\x01\x01\x01', 'hello')

#check that the root is matched with the Ex1
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node

root hash 60ea522a5d21d6e5f16bb42de67409733c9dd5ed321cd34e82e26fe268ebcc62


In [9]:
# use "world" instead of "w"
state.update('\x01\x01\x82\x11', 'world')

print '---------AFTER UPDATE----------'
k, v = state.root_node
#check the encoded key
print '\nhp encoded key, in hex:\n', k.encode('hex')

#check node at the root
print '\nroot node:\n', [k, v.encode('hex')]
print 'key hex escape form:', convert_to_hex_escape(k)

#check root hash
print '\nroot hash:\n', state.root_hash.encode('hex')

---------AFTER UPDATE----------

hp encoded key, in hex:
000101

root node:
['\x00\x01\x01', 'f4a5bad12eed2a953dd05d85101defbaf7eee8de2a1643eda849567058e697bc']
key hex escape form: \x00\x01\x01

root hash:
54405b51e4e94e0e5058bdba7f2cb6bab0fafb7b949590053a92d41a63fbdce2


In [10]:
# print the hash that point to the branch node
print '\nHash hex version:\n', convert_to_hex_escape(v)
print '\nhex version of the hash:\n', v.encode('hex')

#get the real value stored with in the branch node
print '\nbranch node value:\n', convert_to_hex_escape(state._decode_to_node(v))

#check length of the RLP encoded branch node
print '\nlength of the RLP encoded value:\n', len(rlp.encode(state._decode_to_node(v)))

#try hash the above list to check that it is equal to the f4a5bad12eed2a953dd05d85101defbaf7eee8de2a1643eda849567058e697bc
print "\ncheck the hash:\n", utils.sha3(rlp.encode(state._decode_to_node(v))).encode('hex')


Hash hex version:
\xf4\xa5\xba\xd1\x2e\xed\x2a\x95\x3d\xd0\x5d\x85\x10\x1d\xef\xba\xf7\xee\xe8\xde\x2a\x16\x43\xed\xa8\x49\x56\x70\x58\xe6\x97\xbc

hex version of the hash:
f4a5bad12eed2a953dd05d85101defbaf7eee8de2a1643eda849567058e697bc

branch node value:
[['\\x31', 'hello'], '', '', '', '', '', '', '', ['\\x32\\x11', 'world'], '', '', '', '', '', '', '', '']

length of the RLP encoded value:
34

check the hash:
f4a5bad12eed2a953dd05d85101defbaf7eee8de2a1643eda849567058e697bc


# Ex4
add value to the branch node of the above example

In [11]:
#initialize trie like in Ex1
state = trie.Trie('trie_db', trie.BLANK_ROOT)

#add key-value pair (010101, 'hello')
state.update('\x01\x01\x01', 'hello')

#add key-value pair (01018211, 'world')
state.update('\x01\x01\x82\x11', 'world')

k, v = state.root_node
#get the real value stored with in the branch node
print '\nbranch node value:\n', convert_to_hex_escape(state._decode_to_node(v))


branch node value:
[['\\x31', 'hello'], '', '', '', '', '', '', '', ['\\x32\\x11', 'world'], '', '', '', '', '', '', '', '']


In [12]:
#add value "Gluon" to the branch node
state.update('\x01\x01', 'Gluon')

print '---------AFTER UPDATE----------'
k, v = state.root_node

#check node at the root
print '\nroot node:\n', [k, v.encode('hex')]

#get the real value stored with in the branch node
print '\nbranch node value:\n', convert_to_hex_escape(state._decode_to_node(v))

---------AFTER UPDATE----------

root node:
['\x00\x01\x01', '4ba5c3b9b75ae9729d8c64cc7093babbfe77e661a7a0df214ea278c9a00c1aad']

branch node value:
[['\\x31', 'hello'], '', '', '', '', '', '', '', ['\\x32\\x11', 'world'], '', '', '', '', '', '', '', 'Gluon']


# Ex5
try to add two value that make the length of the shared nibbles odd so that the path is shown explicitly

In [22]:
#initialize trie like in Ex1
state = trie.Trie('trie_db', trie.BLANK_ROOT)

#add key-value pair (010101, 'hello')
state.update('\x01\x01\x01', 'hello')

#add key-value pair (01018211, 'world')
state.update('\x01\x01\x82\x11', 'world')

#add value "Gluon" to the branch node
state.update('\x01\x01', 'Gluon')

k, v = state.root_node

#check node at the root
print '\nroot node:\n', [k, v.encode('hex')]

#get the real value stored with in the branch node
print '\nbranch node value:\n', convert_to_hex_escape(state._decode_to_node(v))


root node:
['\x00\x01\x01', '4ba5c3b9b75ae9729d8c64cc7093babbfe77e661a7a0df214ea278c9a00c1aad']

branch node value:
[['\\x31', 'hello'], '', '', '', '', '', '', '', ['\\x32\\x11', 'world'], '', '', '', '', '', '', '', 'Gluon']


In [23]:
#add value "Gluon" to the branch node
state.update('\x01\x01\x4a\x6a\xbf\x54', 'Graviton')
state.update('\x01\x01\x4a\x63\x6e', 'Quark')

print '---------AFTER UPDATE----------'
k, v = state.root_node

#check node at the root
print '\nroot node:\n', [k, v.encode('hex')]

#get the real value stored with in the branch node
print '\nthe first branch node value:\n', convert_to_hex_escape(state._decode_to_node(v))

second_extension_node_hash = state._decode_to_node(v)[4]

#see the value at the 4th index of the first branch node
second_extension_index,  second_extension_node_value = state._decode_to_node(second_extension_node_hash)
print '\nvalue at the second extension node:\n', [convert_to_hex_escape(second_extension_index), second_extension_node_value.encode('hex')]

#branch node stored within the second extension node
print '\nthe second branch node value:\n', convert_to_hex_escape(state._decode_to_node(second_extension_node_value))

---------AFTER UPDATE----------

root node:
['\x00\x01\x01', '3f938aa593f5e78a7f1713a3c318d6bfc40656eaf3ba0c3c79fbe85051979976']

the first branch node value:
[['\\x31', 'hello'], '', '', '', '\\x40\\xf5\\x8a\\x52\\xe4\\xee\\x0b\\x46\\x92\\xdf\\xfc\\x58\\xfb\\xbe\\x01\\xaf\\x44\\x0d\\x5e\\x8c\\xa1\\xb8\\x0a\\x0a\\x7e\\x17\\x0f\\xc1\\x21\\xea\\x2a\\xd1', '', '', '', ['\\x32\\x11', 'world'], '', '', '', '', '', '', '', 'Gluon']

value at the second extension node:
['\\x00\\xa6', 'f4a3750177ae2e5e01b643f56d442aceea3056d21d8213806eec0053ff2735b0']

the second branch node value:
['', '', '', ['\\x20\\x6e', 'Quark'], '', '', '', '', '', '', ['\\x20\\xbf\\x54', 'Graviton'], '', '', '', '', '', '']


# Homework

follow this example https://ethereum.org/en/developers/docs/data-structures-and-encoding/patricia-merkle-trie/

In [15]:
def str_to_hex(s):
    return ''.join(['\\x{:02x}'.format(ord(c)) for c in s])

first_leaf = ('do', 'verb')
second_leaf = ('dog', 'puppy')
third_leaf = ('doge', 'coin')
forth_leaf = ('horse', 'stallion')

In [16]:
# Test
input_str = "do"
print(str_to_hex(input_str))


\x64\x6f
